In [97]:
import json

import requests


RULM_URL = 'https://api.rulm.alexkuk.ru/v1'


class RulmError(Exception):
    pass


def rulm_models():
    return requests.get(f'{RULM_URL}/models').json()


def rulm_tokenize(text, model='saiga-7b-q4'):
    response = requests.post(
        f'{RULM_URL}/tokenize',
        json={
            'text': text,
            'model': model
        }
    )
    if response.status_code != 200:
        raise RulmError(response.text)
        
    return response.json()


def rulm_complete_stream(prompt, model='saiga-7b-q4', max_tokens=128, temperature=0.2):
    response = requests.post(
        f'{RULM_URL}/complete',
        json={
            'prompt': prompt,
            'model': model,
            'max_tokens': max_tokens,
            'temperature': temperature
        },
        stream=True
    )
    if response.status_code != 200:
        raise RulmError(response.text)

    for line in response.iter_lines():
        item = json.loads(line)
        error = item.get('error')
        if error:
            raise RulmError(error)
        yield item


def show_rulm_stream(items):
    buffer = []
    for item in items:
        text = item.get('text')
        prompt_progress = item.get('prompt_progress')
        if text:
            buffer.append(text)
            print(text, flush=True, end='')
        else:
            print(f'{prompt_progress * 100:.0f}%', flush=True, end=' ')
            if prompt_progress == 1:
                print('\n', flush=True)
    return ''.join(buffer)


def rulm_complete(prompt, **kwargs):
    items = rulm_complete_stream(prompt, **kwargs)
    buffer = []
    for item in items:
        if item['text']:
            buffer.append(item['text'])
    return ''.join(buffer)


prompt = '9 мая 2023 года это какой день недели?' * 1000
model = 'saiga-7b-q4'

items = rulm_complete_stream(prompt, model, max_tokens=24, temperature=0)
output = show_rulm_stream(items)

RulmError: n_tokens=15064 > n_ctx=2000

In [43]:
prompt = '9 мая 2023 года это какой день недели?'
model = 'saiga-7b-q4'

items = rulm_complete_stream(prompt, model, max_tokens=64, temperature=0)
output = show_rulm_stream(items)

0% 20% 41% 61% 81% 100% 

9 мая 2023 года будет вторник. <end>

In [3]:
output = rulm_complete(prompt, model=model, max_tokens=64, temperature=0)
print(output)

9 мая 2023 года будет вторник. <end>


In [4]:
for model in rulm_models():
    print(model)
    print(rulm_complete(prompt, model=model, max_tokens=64, temperature=0))
    print()

ru-alpaca-7b-f16
 9 мая 2023 года - вторник.

ru-alpaca-7b-q4
 9 мая 2023 года будет вторник.

saiga-7b-f16
9 мая 2023 года будет вторник. Это второй день недели в году. <end>

saiga-7b-q4
9 мая 2023 года будет вторник. <end>

saiga-7b-v2-f16
9 мая 2023 года будет вторник.</s>

saiga-7b-v2-q4
Выходные данные: 52-й день года.</s>

